In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from torchvision import models


In [2]:
# Set random seed for reproducibility
torch.manual_seed(42)

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load all images from the main directory
full_dataset = datasets.ImageFolder(root='/exchange/dspro01/group3/data/train', transform=transform)

# Define the train-validation split ratio
train_size = int(0.8 * len(full_dataset))  # 80% for training
val_size = len(full_dataset) - train_size  # Remaining 20% for validation

# Split dataset
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

In [3]:
from torchvision.models import EfficientNet_B0_Weights

# Load EfficientNet model (using EfficientNet-B0 as example)
model = models.efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)

# Modify the output layer for 12 classes
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, 12)

In [4]:
import torch.optim as optim
import torch.nn as nn

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

num_epochs = 10  # Adjust based on your needs
best_accuracy = 0.0

for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # Validation phase
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%')
    
    # Save the best model
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), 'best_model.pth')
        print("Best model saved!")


Epoch [1/10], Loss: 0.6737, Accuracy: 83.35%
Best model saved!
Epoch [2/10], Loss: 0.4231, Accuracy: 87.33%
Best model saved!
Epoch [3/10], Loss: 0.3311, Accuracy: 87.38%
Best model saved!
Epoch [4/10], Loss: 0.2806, Accuracy: 87.95%
Best model saved!
Epoch [5/10], Loss: 0.2397, Accuracy: 88.54%
Best model saved!
Epoch [6/10], Loss: 0.2039, Accuracy: 89.11%
Best model saved!
Epoch [7/10], Loss: 0.1710, Accuracy: 89.27%
Best model saved!
Epoch [8/10], Loss: 0.1607, Accuracy: 88.58%
Epoch [9/10], Loss: 0.1384, Accuracy: 89.44%
Best model saved!
Epoch [10/10], Loss: 0.1312, Accuracy: 89.81%
Best model saved!
